In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable

In [4]:
df = pd.read_csv('../data/ETH-USD.csv')

length = 14

In [5]:
def calc_rsi(over: pd.Series, fn_roll: Callable) -> pd.Series:
    # Get the difference in price from previous step
    delta = over.diff()
    # Get ride of the header
    delta = delta[1:] 

    # Make the positive gains (up) and negative gains (down) Series
    up, down = delta.clip(lower=0), delta.clip(upper=0).abs()

    roll_up, roll_down = fn_roll(up), fn_roll(down)
    rs = roll_up / roll_down
    rsi = 100.0 - (100.0 / (1.0 + rs))

    # Avoid division-by-zero if `roll_down` is zero
    # This prevents inf and/or nan values.
    rsi[:] = np.select([roll_down == 0, roll_up == 0, True], [100, 0, rsi])
    rsi.name = 'rsi'

    # Assert range
    valid_rsi = rsi[length - 1:]
    assert ((0 <= valid_rsi) & (valid_rsi <= 100)).all()
    # Note: rsi[:length - 1] is excluded from above assertion because it is NaN for SMA.

    return rsi

In [6]:
# rsi_ema = calc_rsi(close, lambda s: s.ewm(span=length).mean())
# rsi_sma = calc_rsi(close, lambda s: s.rolling(length).mean())|
rsi_rma = calc_rsi(df['Close'], lambda s: s.ewm(alpha=1 / length).mean())  # Approximates TradingView.
df['RSI'] = rsi_rma
df = df.drop([0,1])

# Graph
plt.figure(figsize=(8, 6))
rsi_rma.plot()
plt.legend(['ETH RSI Value'])
plt.show()

In [11]:
df['RSI-status'] = pd.Series(dtype=str)

for index, row in df.iterrows():
    if row['RSI'] > 70:
        df['RSI-status'] = 'Overbought'
    elif row['RSI'] < 30:
        df['RSI-status'] = 'Oversold'
    else:
        df['RSI-status'] = 'Normal'

In [16]:
df.to_csv('ethereum-data.csv')